In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
puzzle_data = pd.read_csv("../full_meta_clustered_singletag.tsv", sep='\t')

In [2]:
#Load the full dataset of puzzle attempts
attempts = pd.read_csv("/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00")

In [5]:
#Replace each 0 in the is_passed field with a -1 for calculating the total won - failed later
attempts['is_passed'] = attempts['is_passed'].replace(0, -1,regex=True)

<h2>Get Average User Success Sample Var in Actual Clusters</h2>

In [4]:
#Filter attempts to include only attempts by players who have played at least 100 games
attempts = attempts.loc[attempts['userGamesPlayed'] >= 100]

In [6]:
#Join cluster labels to puzzles that were clustered 
attempts=pd.merge(attempts,puzzle_data[['tactics_problem_id','cluster','tag']],on='tactics_problem_id', how='inner')

In [8]:
#Filter down to attempts for puzzles that are in the top 20 tags
top_tags = ['Mate in 1', 'Mate in 2', 'Fork / Double Attack', 'Mate in 3+',
       'Hanging Piece', 'Mating Net', 'Pin', 'Vulnerable King',
       'Decoy / Deflection', 'Remove the Defender', 'Discovered Attack',
       'Defense', 'Endgame Tactics', 'Basic Checkmates', 'Back Rank',
       'Sacrifice', 'Pawn Promotion', 'Zwischenzug', 'Trapped Piece',
       'Attacking Castled King']
attempts = attempts.loc[attempts['tag'].isin(top_tags)]

In [12]:
puzzle_data.head()

{'Attacking Castled King',
 'Back Rank',
 'Basic Checkmates',
 'Decoy / Deflection',
 'Defense',
 'Discovered Attack',
 'Endgame Tactics',
 'Fork / Double Attack',
 'Hanging Piece',
 'Mate in 1',
 'Mate in 2',
 'Mate in 3+',
 'Mating Net',
 'Pawn Promotion',
 'Pin',
 'Remove the Defender',
 'Sacrifice',
 'Trapped Piece',
 'Vulnerable King',
 'Zwischenzug'}

In [7]:
#Add expected success for each player using formula for ELO expected score here: 
#https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details
''''
def add_success(row):
    prob_rating = row['ratingProblem']
    rating_user = row['ratingUser']
    rating_diff = prob_rating - rating_user
    return row['is_passed'] - 1/(1 + 10**(rating_diff/400))

attempts['success'] = attempts.progress_apply(lambda row: add_success(row),axis=1)
'''

"'\ndef add_success(row):\n    prob_rating = row['ratingProblem']\n    rating_user = row['ratingUser']\n    rating_diff = prob_rating - rating_user\n    return row['is_passed'] - 1/(1 + 10**(rating_diff/400))\n\nattempts['success'] = attempts.progress_apply(lambda row: add_success(row),axis=1)\n"

In [8]:
'''
#Get average user success for each cluster
opponent_ratings_total = attempts.groupby(['user_hash','cluster'])['ratingProblem'].sum()
num_cluster_games = attempts.groupby(['user_hash','cluster']).nunique()
user_cluster_averaged = attempts.groupby(['user_hash','cluster'])['success'].agg('mean')
#Get the variance of average cluster success and then take the average
avg_cluster_variance = user_cluster_averaged.var(level=0).agg('mean')
print(avg_cluster_variance)
'''

"\n#Get average user success for each cluster\nopponent_ratings_total = attempts.groupby(['user_hash','cluster'])['ratingProblem'].sum()\nnum_cluster_games = attempts.groupby(['user_hash','cluster']).nunique()\nuser_cluster_averaged = attempts.groupby(['user_hash','cluster'])['success'].agg('mean')\n#Get the variance of average cluster success and then take the average\navg_cluster_variance = user_cluster_averaged.var(level=0).agg('mean')\nprint(avg_cluster_variance)\n"

In [13]:
#Calculate performance rating per cluster per user
import time

def time_command(command,criteria):
    print('Running: ' + command)
    start = time.time()
    exec(command)
    print('Took: '  + str(time.time()-start))

def calc_user_performance(time=False,criteria='cluster'):
    commands = ["global opponent_ratings_total; opponent_ratings_total = attempts.groupby(['user_hash',criteria])['ratingProblem'].sum()",
                "global num_cluster_games; num_cluster_games = attempts.groupby(['user_hash',criteria])['user_hash'].count()",
                "global wins_plus_losses; wins_plus_losses = attempts.groupby(['user_hash',criteria])['is_passed'].sum()"]
    for command in commands:
        if time:
            time_command(command,criteria)
        else:
            exec(command)
    return (opponent_ratings_total + 400*wins_plus_losses)/num_cluster_games

In [16]:
#Get variance of performance rating per cluster per user
cluster_performance_user = calc_user_performance(True)
avg_cluster_variance = cluster_performance_user.var(level=0).agg('mean')
print(avg_cluster_variance)

53283.67937626435


In [15]:
performance_user

user_hash                                                         cluster
0000ed89d79d5e4fc767591ef5cafeaacd230d9c5022e578808647f10253b4b4  4          2538.888889
000112a893ef54c6a79bcaaccb54e698ec3b4c46e20f7e4b8fc46e601829a451  1          1956.500000
                                                                  2          1834.597403
                                                                  5          1885.470588
                                                                  10         1375.000000
                                                                  11         1362.000000
                                                                  12         2166.000000
                                                                  13         1972.360000
                                                                  14         1911.070896
                                                                  15         1891.870968
                                    

In [17]:
#Get variance of performance rating per tag per user
tag_performance_user = calc_user_performance(time=True,criteria='tag')
avg_cluster_variance = tag_performance_user.var(level=0).agg('mean')
print(avg_cluster_variance)

Running: global opponent_ratings_total; opponent_ratings_total = attempts.groupby(['user_hash',criteria])['ratingProblem'].sum()
Took: 63.32361674308777
Running: global num_cluster_games; num_cluster_games = attempts.groupby(['user_hash',criteria])['user_hash'].count()
Took: 81.53019905090332
Running: global wins_plus_losses; wins_plus_losses = attempts.groupby(['user_hash',criteria])['is_passed'].sum()
Took: 61.449013233184814
44134.626560228426


In [20]:
len(set(attempts['cluster']))

20

<h2>Get Average User Success Sample Var in Fake Clusters</h2>

In [25]:
#Get number of puzzles in each cluster and create a numpy array of cluster labels so that the number of labels
#for cluster 'a' correspond to the count of cluster a in the original puzzle metadata
cluster_count = puzzle_data.groupby('cluster')['tactics_problem_id'].nunique().values
cluster_list = np.array([])
for i in range(20):
    clust_labels = np.full(cluster_count[i],i+1)
    cluster_list = np.append(cluster_list,clust_labels)
cluster_list = cluster_list.astype(int)

In [26]:
#Shuffle the order of the rows of the original puzzle metadata
puzzle_data_shuffled = puzzle_data.copy().sample(frac=1).reset_index(drop=True)
#Add the clusters to the shuffled puzzles so that the mismatch will create clusters of equal size as the original
#But with the puzzles grouped randomly
puzzle_data_shuffled['cluster'] = cluster_list
#Shuffle again for appearance
puzzle_data_shuffled = puzzle_data_shuffled.sample(frac=1).reset_index(drop=True)

In [27]:
puzzle_data_shuffled.head()

,tactics_problem_id,rating,rd,attempt_count,average_seconds,move_count,tags,fen,tag,cluster
0,28169,1066,45.12,60397,126,4,"Mate in 3+,Mating Net",https://lichess.org/analysis/2kr1r2/p1p3p1/3b3...,Mating Net,16
1,32987,1009,44.06,61868,37,3,Fork / Double Attack,https://lichess.org/analysis/6k1/5p1p/6p1/2p5/...,Fork / Double Attack,4
2,35696,1484,45.02,51055,41,1,"Hanging Piece,Overloading,Pin",https://lichess.org/analysis/r1b1kb1r/pppp1pp1...,Overloading,1
3,157958,400,40.15,4532,40,1,Mate in 1,https://lichess.org/analysis/rn1qkb1r/1b1pnp2/...,Mate in 1,14
4,88305,1130,46.43,62583,52,2,"Basic Checkmates,Mate in 2,Sacrifice,Clearance...",https://lichess.org/analysis/3rr3/6k1/6Np/p1p3...,Clearance Sacrifice,4


In [28]:
#Loop that recreates random baseline clusters and recomputes the average variance of player skills in each cluster
results = []
for i in tqdm(range(3)):
    puzzle_data_shuffled = puzzle_data.copy().sample(frac=1).reset_index(drop=True)
    puzzle_data_shuffled['cluster'] = cluster_list
    attempts.drop(['cluster'],axis=1,inplace=True)
    attempts=pd.merge(attempts,
                      puzzle_data_shuffled[['tactics_problem_id','cluster']],
                      on='tactics_problem_id', 
                      how='inner')
    #Get variance of performance rating per cluster per user
    performance_user = calc_user_performance()
    avg_cluster_variance = performance_user.var(level=0).agg('mean')
    print(avg_cluster_variance)
    results.append(avg_cluster_variance)

48279.21326431261
48996.26945307859
48517.741629443146



In [29]:
print(results)

[48279.21326431261, 48996.26945307859, 48517.741629443146]


In [30]:
print(min(results))
print(sum(results)/3)
print(max(results))

48279.21326431261
48597.741448944784
48996.26945307859
